In [28]:
!wget --no-check http://d28rh4a8wq0iu5.cloudfront.net/ads1/data/chr1.GRCh38.excerpt.fasta

--2023-12-21 13:19:24--  http://d28rh4a8wq0iu5.cloudfront.net/ads1/data/chr1.GRCh38.excerpt.fasta
Resolving d28rh4a8wq0iu5.cloudfront.net (d28rh4a8wq0iu5.cloudfront.net)... 13.225.141.118, 13.225.141.46, 13.225.141.183, ...
Connecting to d28rh4a8wq0iu5.cloudfront.net (d28rh4a8wq0iu5.cloudfront.net)|13.225.141.118|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 810105 (791K) [application/octet-stream]
Saving to: ‘chr1.GRCh38.excerpt.fasta.2’

chr1.GRCh38.excerpt 100%[===================>] 791.12K  --.-KB/s    in 0.04s   

2023-12-21 13:19:25 (21.2 MB/s) - ‘chr1.GRCh38.excerpt.fasta.2’ saved [810105/810105]



In [29]:
def readGenome(filename):
    genome = ''
    with open(filename, 'r') as f:
        for line in f:
            # ignore header line with genome information
            if not line[0] == '>':
                genome += line.rstrip()
    return genome
genome = readGenome("chr1.GRCh38.excerpt.fasta")
print(genome)

...


In [30]:
def naive(p, t):
    occurrences = []
    num_comp = 0
    num_aligments = 0
    for i in range(len(t) - len(p) + 1):  # loop over alignments
        match = True
        num_aligments += 1
        for j in range(len(p)):  # loop over characters
            num_comp += 1
            if t[i+j] != p[j]:  # compare characters
                match = False
                break
        if match:
            occurrences.append(i)  # all chars matched; record
    return occurrences, num_comp, num_aligments

##### Q1: How many alignments does the naive exact matching algorithm try when matching the string
GGCGCGGTGGCTCACGCCTGTAATCCCAGCACTTTGGGAGGCCGAGG (derived from human Alu sequences) to the excerpt of human chromosome 1?  (Don't consider reverse complements.)

In [31]:
p = 'GGCGCGGTGGCTCACGCCTGTAATCCCAGCACTTTGGGAGGCCGAGG'
t = genome
_,_,align = naive(p, t)
print(align)

799954


##### Q2: How many character comparisons does the naive exact matching algorithm try when matching the string
GGCGCGGTGGCTCACGCCTGTAATCCCAGCACTTTGGGAGGCCGAGG(derived from human Alu sequences) to the excerpt of human chromosome 1?  (Don't consider reverse complements.)

In [32]:
p = 'GGCGCGGTGGCTCACGCCTGTAATCCCAGCACTTTGGGAGGCCGAGG'
t = genome
_,compare, _ = naive(p, t)
print(compare)

984143


##### Q3: How many alignments does Boyer-Moore try when matching the string
GGCGCGGTGGCTCACGCCTGTAATCCCAGCACTTTGGGAGGCCGAGG(derived from human Alu sequences) to the excerpt of human chromosome 1?  (Don't consider reverse complements.)

In [33]:
def boyer_moore(p, p_bm, t):
    i = 0
    occurrences = []
    num_char_comp = 0
    num_aligments_tried = 0

    while i < len(t) - len(p) + 1:
        shift = 1
        mismatched = False
        num_aligments_tried += 1

        for j in range(len(p)-1, -1, -1):
            num_char_comp += 1
            if p[j] != t[i+j]:
                skip_bc = p_bm.bad_character_rule(j, t[i+j])
                skip_gs = p_bm.good_suffix_rule(j)
                shift = max(shift, skip_bc, skip_gs)
                mismatched = True
                break

        if not mismatched:
            occurrences.append(i)
            skip_gs = p_bm.match_skip()
            shift = max(shift, skip_gs)
        i += shift

    return occurrences, num_char_comp, num_aligments_tried

In [34]:
!wget --no-check http://d28rh4a8wq0iu5.cloudfront.net/ads1/code/bm_preproc.py

--2023-12-21 13:19:27--  http://d28rh4a8wq0iu5.cloudfront.net/ads1/code/bm_preproc.py
Resolving d28rh4a8wq0iu5.cloudfront.net (d28rh4a8wq0iu5.cloudfront.net)... 13.225.141.118, 13.225.141.46, 13.225.141.183, ...
Connecting to d28rh4a8wq0iu5.cloudfront.net (d28rh4a8wq0iu5.cloudfront.net)|13.225.141.118|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 9400 (9.2K) [application/octet-stream]
Saving to: ‘bm_preproc.py.1’

bm_preproc.py.1     100%[===================>]   9.18K  --.-KB/s    in 0s      

2023-12-21 13:19:28 (104 MB/s) - ‘bm_preproc.py.1’ saved [9400/9400]



In [35]:
from bm_preproc import BoyerMoore
p = 'GGCGCGGTGGCTCACGCCTGTAATCCCAGCACTTTGGGAGGCCGAGG'
p_bm = BoyerMoore(p)
_,_,align = boyer_moore(p, p_bm, genome)
print(align)

127974


##### Q4: Question 4
Index-assisted approximate matching. In practicals, we built a Python class called Index

Implementing an ordered-list version of the k-mer index. The Index class is copied below.

In [36]:
!wget --no-check https://d28rh4a8wq0iu5.cloudfront.net/ads1/code/kmer_index.py

--2023-12-21 13:19:28--  https://d28rh4a8wq0iu5.cloudfront.net/ads1/code/kmer_index.py
Resolving d28rh4a8wq0iu5.cloudfront.net (d28rh4a8wq0iu5.cloudfront.net)... 13.225.141.118, 13.225.141.46, 13.225.141.183, ...
Connecting to d28rh4a8wq0iu5.cloudfront.net (d28rh4a8wq0iu5.cloudfront.net)|13.225.141.118|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 971 [application/octet-stream]
Saving to: ‘kmer_index.py.1’

kmer_index.py.1     100%[===================>]     971  --.-KB/s    in 0s      

2023-12-21 13:19:29 (385 MB/s) - ‘kmer_index.py.1’ saved [971/971]



In [37]:
import bisect
class Index(object):
    def __init__(self, t, k):
        ''' Create index from all substrings of size 'length' '''
        self.k = k  # k-mer length (k)
        self.index = []
        for i in range(len(t) - k + 1):  # for each k-mer
            self.index.append((t[i:i+k], i))  # add (k-mer, offset) pair
        self.index.sort()  # alphabetize by k-mer

    def query(self, p):
        ''' Return index hits for first k-mer of P '''
        kmer = p[:self.k]  # query with first k-mer
        i = bisect.bisect_left(self.index, (kmer, -1))  # binary search
        hits = []
        while i < len(self.index):  # collect matching index entries
            if self.index[i][0] != kmer:
                break
            hits.append(self.index[i][1])
            i += 1
        return hits

##### Write a function that, given a length-24 pattern P and given an
Index
Index object built on 8-mers, finds all approximate occurrences of P within T with up to 2 mismatches. Insertions and deletions are not allowed. Don't consider any reverse complements.

How many times does the string
GGCGCGGTGGCTCACGCCTGTAAT, which is derived from a human Alu sequence, occur with up to 2 substitutions in the excerpt of human chromosome 1?  (Don't consider reverse complements here.)

Hint 1: Multiple index hits might direct you to the same match multiple times, but be careful not to count a match more than once.

Hint 2: You can check your work by comparing the output of your new function to that of the
naive_2mm function implemented in the previous module.

In [38]:
def approximate_match_index(p, t, k):
    segment_length = round(len(p) // (k+1))
    hits = 0
    all_matches = set()
    index = Index(t, 8) # built on 8-mers
    for i in range(k+1):
        start = i * segment_length
        end = min((i+1) * segment_length, len(p))
        matches = index.query(p[start:end])
        hits += len(matches)
        for m in matches:
            text_offset = m - start
            if text_offset < 0 or (text_offset + len(p)) > len(t):
                continue
            mismatches = 0
            for j in range(0, start):
                if not p[j] == t[text_offset + j]:
                    mismatches += 1
                    if mismatches > k:
                        break
            for j in range(end, len(p)):
                if not p[j] == t[text_offset + j]:
                    mismatches += 1
                    if mismatches > k:
                        break
            if mismatches <= k:
                all_matches.add(text_offset)
    return list(all_matches), hits

In [40]:
t = genome
p = 'GGCGCGGTGGCTCACGCCTGTAAT'
k = 2
ans, hits = approximate_match_index(p, t, k)
print("occurrences: " + str(len(ans)))

occurrences: 19


##### Q5: Using the instructions given in Question 4, how many total index hits are there when searching for occurrences of
GGCGCGGTGGCTCACGCCTGTAAT with up to 2 substitutions in the excerpt of human chromosome 1?

  (Don't consider reverse complements.)

Hint: You should be able to use the
boyer_moore function (or the slower
naive function) to double-check your answer.

In [41]:
print("index hits: " + str(hits))

index hits: 90


##### Q6: Question 6
Let's examine whether there is a benefit to using an index built using subsequences of T rather than substrings, as we discussed in the "Variations on k-mer indexes" video.  We'll consider subsequences involving every N characters.  For example, if we split
ATATAT into two substring partitions, we would get partitions
ATA(the first half) and
TAT(second half).  But if we split
ATATATinto two  subsequences  by taking every other character, we would get
AAA(first, third and fifth characters) and
TTT(second, fourth and sixth).

Another way to visualize this is using numbers to show how each character of P is allocated to a partition.  Splitting a length-6 pattern into two substrings could be represented as
111222, and splitting into two subsequences of every other character could be represented as
121212

The following class
SubseqIndex is a more general implementation of
Index that additionally handles subsequences. It only considers subsequences that take every Nth character:

In [42]:
import bisect

class SubseqIndex(object):
    """ Holds a subsequence index for a text T """

    def __init__(self, t, k, ival):
        """ Create index from all subsequences consisting of k characters
            spaced ival positions apart.  E.g., SubseqIndex("ATAT", 2, 2)
            extracts ("AA", 0) and ("TT", 1). """
        self.k = k  # num characters per subsequence extracted
        self.ival = ival  # space between them; 1=adjacent, 2=every other, etc
        self.index = []
        self.span = 1 + ival * (k - 1)
        for i in range(len(t) - self.span + 1):  # for each subseq
            self.index.append((t[i:i+self.span:ival], i))  # add (subseq, offset)
        self.index.sort()  # alphabetize by subseq

    def query(self, p):
        """ Return index hits for first subseq of p """
        subseq = p[:self.span:self.ival]  # query with first subseq
        i = bisect.bisect_left(self.index, (subseq, -1))  # binary search
        hits = []
        while i < len(self.index):  # collect matching index entries
            if self.index[i][0] != subseq:
                break
            hits.append(self.index[i][1])
            i += 1
        return hits

Write a function that, given a length-24 pattern P and given a
SubseqIndex
SubseqIndex object built with k = 8 and ival = 3, finds all approximate occurrences of P within T with up to 2 mismatches.

When using this function, how many total index hits are there when searching for
GGCGCGGTGGCTCACGCCTGTAATwith up to 2 substitutions in the excerpt of human chromosome 1?  (Again, don't consider reverse complements.)

In [43]:
def approximate_match_subseq_index(p, t, k):
    segment_length = round(len(p) // (k+1))
    hits = 0
    all_matches = set()
    index = SubseqIndex(t, 8, 3) # built on 8-mers and subsequence intervals of 3
    for start in range(k+1):
        matches = index.query(p[start:])
        hits += len(matches)
        for m in matches:
            text_offset = m - start
            if text_offset < 0 or (text_offset + len(p)) > len(t):
                continue
            mismatches = 0
            for j in range(0, len(p)):
                if not p[j] == t[text_offset + j]:
                    mismatches += 1
                    if mismatches > k:
                        break
            if mismatches <= k:
                all_matches.add(text_offset)
    return list(all_matches), hits

In [44]:
t = genome
p = 'GGCGCGGTGGCTCACGCCTGTAAT'
k = 2
_, hits = approximate_match_subseq_index(p, t, k)
print(hits)

79
